# Machine Learning Essentials SS25 - Exercise Sheet 7

## Instructions
- `TODO`'s indicate where you need to complete the implementations.
- You may use external resources, but <b>write your own solutions</b>.
- Provide concise, but comprehensible comments to explain what your code does.
- Code that's unnecessarily extensive and/or not well commented will not be scored.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import dok_matrix, coo_matrix
from scipy.sparse.linalg import lsqr

DATA_DIR = "/hs_tomography"  # Change this to your data directory

## Task 1


In [28]:
size = 77
sensor_size = 109
projection_angles = 90


alphas = np.load(f'./{DATA_DIR}/alphas_{size}.npy')
X_example = np.load(f'./{DATA_DIR}/X_example.npy')

In [29]:
X_example.shape

(45, 100)

In [128]:
def construct_X(M, alphas, Np=None):
    # TODO: implement vectorised sparse projection matrix X, according to the instructions on the sheet. Remember to vectorize everything (no for-loops over pixels) and use the mentioned matrix formats (COOrdinate/Compressed Sparse Column). The large case (`M=195`) should build in approx. < 10s on a laptop.
    if Np==None:
        Np = int(np.ceil(np.sqrt(2)*M))
    
    D = M*M
    

    # C matrix with pixel centers of shape (2, D)

    a, b = np.mgrid[-(M-1)/2:(M-1)/2+1, (M-1)/2:-(M-1)/2-1:-1]
    C = np.vstack([a.ravel(order='F'), b.ravel(order='F')])

    # unit vectors
    alphas = np.deg2rad(alphas)  
    unit_vectors = np.array([np.cos(alphas), np.sin(alphas)])
    s0 = (Np-1)/2

    row_indices = []
    col_indices = []
    weights = []

    for i, alpha in enumerate(alphas):
        
        n = unit_vectors[:, i].reshape(2, 1)

        P = (n.T @ C).flatten() + s0

        idx_left = np.floor(P).astype(int)
        idx_right = idx_left + 1

        frac = P - idx_left

        w0 = (1 - frac)
        w1 = frac

        mask0 = (idx_left >= 0) & (idx_left < Np)
        mask1 = (idx_right >= 0) & (idx_right < Np)

        row0 = idx_left[mask0] + i * Np
        row1 = idx_right[mask1] + i * Np

        col0 = np.where(mask0)[0]
        col1 = np.where(mask1)[0]

        row_indices.append(row0)
        row_indices.append(row1)
        col_indices.append(col0)
        col_indices.append(col1)
        weights.append(w0[mask0])
        weights.append(w1[mask1])

    if weights:
        row_indices = np.concatenate(row_indices)
        col_indices = np.concatenate(col_indices)
        weights = np.concatenate(weights)

    else:
        row_indices = np.array([], dtype=int)
        col_indices = np.array([], dtype=int)
        weights = np.array([], dtype=np.float32)

    return coo_matrix((weights, (row_indices, col_indices)), shape=(len(alphas) * Np, D), dtype=np.float32)


In [132]:
X = construct_X(10, np.array([-33, 1, 42])).toarray()


### Quick sanity check 


In [133]:
# TODO: Check if your image matches `X_example.npy` (Figure 2) up to mirror / transpose.
print(np.allclose(X, X_example, atol=1e-5))

True


## Task 2 – Reconstruct the tomogram

In [13]:
# TODO: Reconstruct the tomogram and plot it as a 2D image. Use scipy.sparse.linalg.lqsr() to make use of the sparsity of your matrix, see the instructions on the sheet.


### Diagnosis
TODO: describe the anomaly and suggest a treatment.


## Task 3 

In [14]:
def reconstruct_with_subset():
    # TODO: Reconstruct X using only a subset of projection angles.
    raise NotImplementedError

# TODO: Reduce the number of projectio angles in a sensible way and visualize how the reconstruction is affected by the number of angles used.

TODO: state the smallest number of projections that still resolves the pathology clearly enough to give a diagnosis and propose a treatment.
